In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'PHI_2') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None, min_samples_leaf=2, min_samples_split=5, n_estimators=100),
        lags=14
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,30,33.0,30.0
1,HOME_&_GARDEN_1_027,78,95.0,94.0
2,HOME_&_GARDEN_1_053,61,56.0,40.0
3,HOME_&_GARDEN_1_140,29,30.0,30.0
4,HOME_&_GARDEN_1_177,63,122.0,78.0
...,...,...,...,...
95,SUPERMARKET_3_499,169,160.0,164.0
96,ACCESORIES_1_158,137,110.0,102.0
97,SUPERMARKET_3_282,484,335.0,335.0
98,ACCESORIES_1_354,63,41.0,30.0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

28026
24640.0
3386.0


In [8]:
tienda = pd.DataFrame(resultados)
file_name = "PHI_2_unidades.xlsx"
tienda.to_excel(file_name)

In [4]:
pd.DataFrame(resultados)

,producto,prediccion,test
0,ACCESORIES_1_108,"[0.45242103580562654, 0.22019047619047616, 0.5...","[0, 1, 1, 1, 1, 2, 0, 1, 3, 0, 2, 0, 0, 0, 3, ..."
1,HOME_&_GARDEN_1_027,"[1.8732579365079365, 2.830400793650793, 1.5652...","[1, 5, 0, 1, 0, 2, 1, 4, 1, 1, 1, 1, 0, 0, 1, ..."
2,HOME_&_GARDEN_1_053,"[3.0172088744588734, 4.094804473304472, 3.1821...","[3, 3, 5, 2, 2, 0, 3, 3, 8, 1, 4, 2, 1, 4, 1, ..."
3,HOME_&_GARDEN_1_140,"[2.4839126984126985, 2.2182579365079373, 1.563...","[3, 4, 0, 1, 5, 1, 2, 1, 5, 0, 4, 2, 3, 2, 5, ..."
4,HOME_&_GARDEN_1_177,"[2.5805000000000002, 1.8433816738816733, 1.160...","[2, 3, 5, 0, 1, 1, 2, 1, 2, 2, 2, 2, 0, 0, 0, ..."
...,...,...,...
95,SUPERMARKET_3_499,"[11.283150793650796, 13.517007936507934, 11.98...","[12, 8, 2, 6, 2, 10, 7, 5, 4, 9, 8, 1, 10, 14,..."
96,ACCESORIES_1_158,"[4.742321428571427, 5.657997474747475, 4.08998...","[10, 4, 10, 6, 7, 6, 2, 3, 6, 4, 3, 9, 4, 1, 5..."
97,SUPERMARKET_3_282,"[16.755575757575755, 25.587550505050505, 17.20...","[15, 31, 15, 11, 9, 21, 15, 7, 7, 8, 12, 12, 1..."
98,ACCESORIES_1_354,"[1.5083095238095239, 3.907556166056165, 3.3659...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, ..."


## Predigo 30 dias(los de test) 